In [1]:
import pandas as pd 
import numpy as np
import os
from subprocess import check_output
import shutil
import rdflib
from multiprocessing import Process, Manager, Queue
import multiprocessing

from rule import *
from amie import *
from experiment import *

In [2]:
root_source_FB = "./../../FB15k_mail/"
root_source_DB = "./../../DB15k_mail/"

root_data_FB = "./../../Data_mail/FB_LTE_GT_"
root_data_DB = "./../../Data_mail/DB_LTE_GT_"

# FB15K 

### Prepare the data

As we need a baseline for Amie, we are not going to change the value of anything. 

In [3]:
data = open(root_source_FB+"numericals.txt", "r")

numerical_predicate = set()

for predicate in data:
    numerical_predicate.add(predicate.split("\n")[0])
    
data.close()

In [4]:
thresholds = np.arange(0.25,1,0.25) #[0.25, 0.50, 0.75]
thresholds_str = ["-"+(str(int(i*100))) for i in thresholds]

In [5]:
def write_file(X, f, threshold, comparator):
    f.write(f"<http:{X['Subject']}>\t{X['Predicate'][:-1]}_LTE_{threshold}>\t<http:/{X['Object']<=comparator}>\n")
    f.write(f"<http:{X['Subject']}>\t{X['Predicate'][:-1]}_GT_{threshold}>\t<http:/{X['Object']>comparator}>\n")

In [6]:
data = open(root_source_FB+"train.txt", "r")
f = open(root_data_FB+"train.tsv", "w")

dic_predicate = {}

for line in data:
    line_split = line.split("\n")[0].split("\t")
    if line_split[1] in numerical_predicate:
        line_split[2] = float(line_split[2]) 
        line_split = tuple(line_split)
        if line_split[1] in dic_predicate.keys():
            dic_predicate[line_split[1]].add(line_split)
        else : 
            dic_predicate[line_split[1]] = {line_split}
    else:
        for var in line_split:
            f.write('<http:'+var+'>\t')
        f.write("\n")
    
data.close()
f.close()

In [7]:
f = open(root_data_FB+"train.tsv", "a")

for key in dic_predicate.keys():
    tp_df = pd.DataFrame.from_dict(dic_predicate[key]).rename(columns={0: "Subject", 1: "Predicate", 2: "Object"})
    tp_df_describe = tp_df["Object"].quantile(thresholds)
    for threshold in thresholds: 
        tp_df.apply(write_file, args=(f, threshold, tp_df_describe[threshold]), axis=1)

f.close()

### Launch Amie

In [8]:
res = check_output(f'java -jar ./../amie3.jar {root_data_FB+"train.tsv"}', shell=True)

res_parsed = parse_amie(res)

Using the default schema relations


In [9]:
len(res_parsed)

339508

In [10]:
# res_parsed

### Clean rules

In [11]:
res_parsed_clean = res_parsed.copy()
for i in res_parsed:
    if len(i.hypotheses) == 1:
        if (i.hypotheses[0].predicate.split("LTE")[0] == i.conclusion.predicate.split("LTE")[0]) and len(i.hypotheses[0].predicate.split("LTE")) == 2:
#             print(i)
            res_parsed_clean.remove(i)

In [12]:
len(res_parsed_clean)

339358

### Number of numericals in the rules

In [13]:
num_f = open("LTE_GT_num_rules.txt", "w")
symb_f = open("LTE_GT_symb_rules.txt", "w")

def predicate_is_numerical(atom, numerical_predicate):
    return atom.predicate in numerical_predicate

rule_with_numerical_in_hyp = 0
rule_with_numerical_in_conc = 0
rule_with_numerical = 0

for rule in res_parsed:
    num = False
    for hyp in rule.hypotheses:
        if predicate_is_numerical(hyp, numerical_predicate):
            rule_with_numerical_in_hyp+=1
            num = True
            break
    if predicate_is_numerical(rule.conclusion, numerical_predicate):
        rule_with_numerical_in_conc+=1
        num=True
    if num:
        rule_with_numerical += 1
        num_f.write(str(rule)+"\n")
    else:
        symb_f.write(str(rule)+"\n")
    
print("Rule with numerical : ", rule_with_numerical)
print("Rule without numerical : ", len(res_parsed) - rule_with_numerical)
print("Rule with numerical in hypotheses : ", rule_with_numerical_in_hyp)
print("Rule with numerical in conclusion : ", rule_with_numerical_in_conc)

num_f.close()
symb_f.close()

Rule with numerical :  0
Rule without numerical :  339508
Rule with numerical in hypotheses :  0
Rule with numerical in conclusion :  0


### Test the rules through test

In [14]:
data = open(root_data_FB+"train.tsv", "r")
f = open(root_data_FB+"train_rdflib.nt", "w")

for line in data:
    f.write(line.split("\n")[0]+" . \n")

data.close()
f.close()

In [15]:
data = open(root_source_FB+"test.txt", "r")

set_instances_to_predict = set()

for line in data:
    line_splited = line.split("\n")[0].split("\t")
    for i in range(len(line_splited)):
        line_splited[i] = "<http:"+line_splited[i]+">"
    set_instances_to_predict.add(tuple(line_splited))
        
data.close()

In [16]:
print(len(set_instances_to_predict))

59250


In [17]:
rules_associated_to_query = {}

for rule in res_parsed:
    if rule.conclusion.predicate in rules_associated_to_query.keys():
        rules_associated_to_query[rule.conclusion.predicate].add(rule)
    else:
        rules_associated_to_query[rule.conclusion.predicate] = set()
        rules_associated_to_query[rule.conclusion.predicate].add(rule)

In [18]:
g = rdflib.Graph()
g.parse(root_data_FB+"train_rdflib.nt")

<Graph identifier=N030ef3ef5b9b437ab6c850a3aad3654c (<class 'rdflib.graph.Graph'>)>

In [19]:
def print_rules_nicely(rule):
    toprint = ""
    for hyp in rule.hypotheses:
        toprint += hyp.predicate+" & "
    toprint = toprint[:-2]
    toprint += "=>"+rule.conclusion.predicate
    print(toprint)

In [ ]:
%%time

def predict_instance(name, g, queue, prediction_per_instance_man, rules_associated_to_query, cpt, total_length, print_advancment):
    
    print(f"Process n°{name} : Launched")
    
    while not queue.empty():
        
        instance = queue.get()
        
        dict_tp = {}
        
        if instance[1] in rules_associated_to_query.keys():
            for rule in rules_associated_to_query[instance[1]]:
                try:
                    qres = g.query(create_query(rule, instance[2]))

                    set_res = set()
                    bool_res = False
                    for row in qres:
                        bool_res = True
                        set_res.add(str(row.a))

                    if bool_res:
                        dict_tp[rule] = set_res
            
                except:
                    print(create_query(rule, instance[2]))
                    
            prediction_per_instance_man[instance] = dict_tp
            
        else:
            prediction_per_instance_man[instance] = {}
        
        cpt.value += 1
        if (cpt.value/total_length > print_advancment.value):
            print(cpt)
            print_advancment.value+=0.1
        
    print(f"Process n°{name} : Finished")   
    
q = Queue()
prediction_per_instance = {}

for instance in list(set_instances_to_predict):
    q.put(instance)

size_queue = q.qsize()

print("Queue finished")

with Manager() as manager:

    processes_to_create = multiprocessing.cpu_count()
    processes = list()

    prediction_per_instance_man = manager.dict()
    cpt = manager.Value("cpt",0)
    print_advancment = manager.Value("print_advancment",0)

    for name in range(processes_to_create):
        x = Process(target=predict_instance, args=(name, g, q, prediction_per_instance_man, rules_associated_to_query, cpt, size_queue, print_advancment))
        processes.append(x)
        x.start()
        
    for index, process in enumerate(processes):
        process.join()

    prediction_per_instance = prediction_per_instance_man.copy()

In [ ]:
len(prediction_per_instance.keys())

In [ ]:
df_prediction = {}

for prediction_instance in prediction_per_instance:
    df_rules = {}
    for rule in prediction_per_instance[prediction_instance]:
        df_rules[rule] = [set(prediction_per_instance[prediction_instance][rule]), rule.stdConfidence, rule.pcaConfidence]
    df_prediction[prediction_instance] = pd.DataFrame.from_dict(df_rules, orient="index", columns=["Prediction", "Std Confidence", "Pca Confidence"])

In [ ]:
print("----- Democracy -----")
hit_at(df_prediction, democracy, 2)
hit_at(df_prediction, democracy, 5)
hit_at(df_prediction, democracy, 10)
hit_at(df_prediction, democracy, 1000)

print("----- Expert -----")
hit_at(df_prediction, expert, 2)
hit_at(df_prediction, expert, 5)
hit_at(df_prediction, expert, 10)
hit_at(df_prediction, expert, 1000)

In [ ]:
%%time

def predict_instance(name, g, instances, prediction_per_instance_man, rules_associated_to_query, cpt, total_length, print_advancment):
    
    print(f"Process n°{name} : Launched")
    
    for instance in instances:
        
        dict_tp = {}
        if instance[1] in rules_associated_to_query.keys():
            for rule in rules_associated_to_query[instance[1]]:
                try:
                    qres = g.query(create_query(rule, instance[2]))
                    tp = create_query(rule, instance[2])
                    if ("LTE"  in tp) or ("GT" in tp):
                        print(tp)

                    set_res = set()
                    bool_res = False
                    for row in qres:
                        bool_res = True
                        set_res.add(str(row.a))

                    if bool_res:
                        dict_tp[rule] = set_res
            
                except:
                    print(create_query(rule, instance[2]))
                    
            prediction_per_instance_man[instance] = dict_tp
            
        else:
            prediction_per_instance_man[instance] = {}
        
        cpt.value += 1
        if (cpt.value/total_length > print_advancment.value):
            print(cpt)
            print_advancment.value+=0.1
        
    print(f"Process n°{name} : Finished")

prediction_per_instance_test = {}

size_queue = len(list(set_instances_to_predict)[:100])

with Manager() as manager:

    processes_to_create = 1#multiprocessing.cpu_count()-3
    processes = list()

    prediction_per_instance_man = manager.dict()
    cpt = manager.Value("cpt",0)
    print_advancment = manager.Value("print_advancment",0)
    
    instances_list = list(set_instances_to_predict)[:100]

    for name in range(processes_to_create):
        x = Process(target=predict_instance, args=(name, g, instances_list[int(np.floor(name*len(instances_list)/processes_to_create)): int(np.floor((name+1)*len(instances_list)/processes_to_create))], 
                                                   prediction_per_instance_man, rules_associated_to_query, cpt, size_queue, print_advancment))
        processes.append(x)
        x.start()
        
    for index, process in enumerate(processes):
        process.join()

    prediction_per_instance_test = prediction_per_instance_man.copy()

# DB15K 

### Prepare the data

As we need a baseline for Amie, we are not going to change the value of anything. 

In [ ]:
data = open(root_source_DB+"numericals.txt", "r")

numerical_predicate = set()

for predicate in data:
    numerical_predicate.add(predicate.split("\n")[0])
    
data.close()

In [ ]:
thresholds = np.arange(0.25,1,0.25) #[0.25, 0.50, 0.75]
thresholds_str = ["-"+(str(int(i*100))) for i in thresholds]

In [ ]:
data = open(root_source_DB+"train.txt", "r")
f = open(store_data_DB, "w")

dic_predicate = {}

for line in data:
    line_split = line.split("\n")[0].split(" ")
    if len(line_split) == 1:
        line_split = line.split("\n")[0].split("\t")
    if line_split[1] in numerical_predicate:
        line_split[2] = float(line_split[2]) 
        line_split = tuple(line_split)
        if line_split[1] in dic_predicate.keys():
            dic_predicate[line_split[1]].add(line_split)
        else : 
            dic_predicate[line_split[1]] = {line_split}
    else:
        f.write(line)
    
data.close()
f.close()

In [ ]:
f = open(store_data_DB, "a")

for key in dic_predicate.keys():
    tp_df = pd.DataFrame.from_dict(dic_predicate[key]).rename(columns={0: "Subject", 1: "Predicate", 2: "Object"})
    tp_df_describe = tp_df["Object"].quantile(thresholds)
    for threshold in thresholds: 
        tp_df.apply(write_file, args=(f, threshold, tp_df_describe[threshold]), axis=1)

f.close()

### Launch Amie

In [ ]:
res = check_output(f'java -jar ./../amie3.jar {store_data_DB}', shell=True)

res_parsed = parse_amie(res)

In [ ]:
len(res_parsed)

In [ ]:
res_parsed

### Clean rules

In [ ]:
res_parsed_clean = res_parsed.copy()
for i in res_parsed:
    if len(i.hypotheses) == 1:
        if (i.hypotheses[0].predicate.split("LTE")[0] == i.conclusion.predicate.split("LTE")[0]) and len(i.hypotheses[0].predicate.split("LTE")) == 2:
            print(i)
            res_parsed_clean.remove(i)

In [ ]:
len(res_parsed_clean)

### Number of numericals in the rules

In [ ]:
def predicate_is_numerical(atom):
    return "LTE" in atom.predicate or "GT" in atom.predicate or atom.objectD.isdigit()

rule_with_numerical_in_hyp = 0
rule_with_numerical_in_conc = 0
rule_with_numerical = 0

for rule in res_parsed_clean:
    num = False
    for hyp in rule.hypotheses:
        if predicate_is_numerical(hyp):
            rule_with_numerical_in_hyp+=1
            num = True
            break
    if predicate_is_numerical(rule.conclusion):
        rule_with_numerical_in_conc+=1
        num=True
    if num:
        rule_with_numerical += 1
    
print("Rule with numerical : ", rule_with_numerical)
print("Rule without numerical : ", len(res_parsed_clean) - rule_with_numerical)
print("Rule with numerical in hypotheses : ", rule_with_numerical_in_hyp)
print("Rule with numerical in conclusion : ", rule_with_numerical_in_conc)